Problem 1
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that - M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the - neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [ ]:
import numpy as np
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source=requests.get(url).text

In [ ]:
soup=BeautifulSoup(source,'xml')

In [ ]:
table=soup.find('table')

In [ ]:
# create dataframe 
columns_names=['Postalcode','Borough','Neighborhood']
df=pd.DataFrame(columns=columns_names)

In [ ]:
# searcg all the postcode, borough, neighborhood
for tr_cell in table.find_all('tr'):
  row_data=[]
  for td_cell in tr_cell.find_all('td'):
    row_data.append(td_cell.text.strip())
  if len(row_data)==3:
    df.loc[len(df)]=row_data

In [ ]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [ ]:
# data cleaning
# remove rows where Borough is ''Not assigned

df=df[df['Borough']!='Not assigned']

In [ ]:
df.loc[df.Neighborhood=='Not assigned','Neighborhood']=df.loc[df.Neighborhood=='Not assigned','Borough']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [ ]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [ ]:
df_merge=pd.merge(df,temp_df,on='Postalcode')

In [ ]:
df_merge.drop('Neighborhood',axis=1,inplace=True)

In [ ]:
df_merge.drop_duplicates(inplace=True)

In [ ]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [ ]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [ ]:
df_merge.shape

(103, 3)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In [ ]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Geospatial_Coordinates.csv to Geospatial_Coordinates.csv


In [ ]:
geo_df=pd.read_csv('Geospatial_Coordinates.csv')

In [ ]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merge=df_merge.merge(geo_df,on='Postalcode')

In [ ]:
geo_merge.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [ ]:
geo_data=geo_merge
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


**Explore and cluster the neighborhoods in Toronto**. 

You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make. 

2. to generate maps to visualize your neighborhoods and how they cluster together. 


Getting all the rows from the data frame which contains Torontoin Borough

In [ ]:
df1=geo_data[geo_data['Borough'].str.contains('Toronto',regex=False)]
df1.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


visualizing all the neighborhoods of the above data frame using Folium

In [ ]:
import folium # plotting library
latitude=43.651070
longitude=-79.347015

map_toronto=folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lng,borough,neighborhood in zip(df1['Latitude'],df1['Longitude'],df1['Borough'],df1['Neighborhood']):
  label='{}, {}'.format(neighborhood,borough)
  label=folium.Popup(label,parse_html=True)
  folium.CircleMarker(
      [lat,lng],
      radius=5,
      popup=label,
      color='blue',
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7,
      parse_html=False
  ).add_to(map_toronto)

map_toronto

using KMeans clustering for the clustering of the neighborhoods

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

ncluster=5
X=df1.drop(['Postalcode','Borough','Neighborhood'],1)
kmeans=KMeans(n_clusters=ncluster,random_state=0).fit(X)
kmeans.labels_

df1.insert(0,'Cluster Labels',kmeans.labels_)


In [ ]:
df1.head()

,Cluster Labels,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,1,M4E,East Toronto,The Beaches,43.676357,-79.293031


visualizing neighborhood with different colors for different cluster labels

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
xs = np.arange(ncluster)
ys = [i + xs + (i*xs)**2 for i in range(ncluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df1['Latitude'], df1['Longitude'], df1['Neighborhood'], df1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters